In [0]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
import os
import json

warnings.filterwarnings('ignore')

# =============================================================================
# SETUP
# =============================================================================

MAPS_DIR = 'maps'
if not os.path.exists(MAPS_DIR):
    os.makedirs(MAPS_DIR)
    print(f"Created directory: {MAPS_DIR}/\n")

BOROUGH_COLORS = {
    'Manhattan': '#E63946',
    'Brooklyn': '#457B9D',
    'Queens': '#2A9D8F',
    'Bronx': '#9B59B6',
    'Staten Island': '#F4A261'
}

# =============================================================================
# LOAD DATA
# =============================================================================

print("="*80)
print("🗺️ CREATING INTERACTIVE MAPS WITH MOVABLE STATS PANEL")
print("="*80)

df = pd.read_csv('nyc_housing_processed.csv')
df_map = df[df['latitude'].notna() & df['longitude'].notna()].copy()
print(f"Loaded {len(df_map):,} properties with coordinates\n")

df_res = pd.read_csv('nyc_housing_residential_units.csv')
df_res_map = df_res[df_res['latitude'].notna() & df_res['longitude'].notna()].copy()
print(f"Residential units with coordinates: {len(df_res_map):,}\n")

print("Creating interactive maps with movable stats panel...")

# =============================================================================
# COMPACT MAP SETTINGS
# =============================================================================

MAP_WIDTH = 1100
MAP_HEIGHT = 650

# =============================================================================
# CUSTOM HTML TEMPLATE WITH MOVABLE SELECTION STATS
# =============================================================================

def create_map_with_selection_stats(fig, filename, data_for_stats):
    """
    Create an interactive map with box/lasso select and MOVABLE stats display.
    
    The stats panel can be dragged anywhere on the screen!
    
    FIXED: Uses single trace approach so point indices match data array indices.
    """
    
    # Reset index to ensure JavaScript array indices match point indices
    data_for_stats = data_for_stats.reset_index(drop=True)
    
    # Convert data to JSON for JavaScript
    stats_data = data_for_stats.to_dict('records')
    
    # Get the plotly figure as HTML div
    fig_html = fig.to_html(
        full_html=False,
        include_plotlyjs='cdn',
        div_id='plotly-map',
        config={
            'scrollZoom': True,
            'displayModeBar': True,
            'displaylogo': False,
            'modeBarButtonsToAdd': ['select2d', 'lasso2d'],
            'modeBarButtonsToRemove': [],
        }
    )
    
    # Create full HTML with custom JavaScript for selection handling AND dragging
    full_html = f'''
<!DOCTYPE html>
<html>
<head>
    <title>NYC Housing Map - Interactive Selection</title>
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        body {{
            font-family: 'Segoe UI', Arial, sans-serif;
            background: #0D1117;
            color: white;
            padding: 10px;
            overflow: hidden;
        }}
        .map-container {{
            width: 100%;
            height: calc(100vh - 20px);
        }}
        
        /* MOVABLE STATS PANEL */
        .stats-panel {{
            position: fixed;
            top: 80px;
            right: 20px;
            width: 300px;
            background: #161B22;
            border: 2px solid #30363D;
            border-radius: 12px;
            box-shadow: 0 10px 40px rgba(0,0,0,0.5);
            z-index: 1000;
            overflow: hidden;
            transition: box-shadow 0.3s;
        }}
        .stats-panel:hover {{
            box-shadow: 0 15px 50px rgba(0,0,0,0.7);
        }}
        .stats-panel.dragging {{
            box-shadow: 0 20px 60px rgba(230, 57, 70, 0.4);
            border-color: #E63946;
        }}
        
        /* DRAG HANDLE */
        .drag-handle {{
            background: linear-gradient(135deg, #E63946 0%, #C53030 100%);
            padding: 12px 15px;
            cursor: grab;
            display: flex;
            justify-content: space-between;
            align-items: center;
            user-select: none;
        }}
        .drag-handle:active {{
            cursor: grabbing;
        }}
        .drag-handle-title {{
            font-size: 16px;
            font-weight: bold;
            color: white;
            display: flex;
            align-items: center;
            gap: 8px;
        }}
        .drag-handle-icon {{
            font-size: 14px;
            opacity: 0.8;
        }}
        .drag-dots {{
            display: flex;
            flex-direction: column;
            gap: 3px;
            opacity: 0.6;
        }}
        .drag-dots div {{
            display: flex;
            gap: 3px;
        }}
        .drag-dots span {{
            width: 4px;
            height: 4px;
            background: white;
            border-radius: 50%;
        }}
        
        /* MINIMIZE BUTTON */
        .minimize-btn {{
            background: rgba(255,255,255,0.2);
            border: none;
            color: white;
            width: 28px;
            height: 28px;
            border-radius: 5px;
            cursor: pointer;
            font-size: 16px;
            display: flex;
            align-items: center;
            justify-content: center;
            transition: all 0.3s;
        }}
        .minimize-btn:hover {{
            background: rgba(255,255,255,0.3);
            transform: scale(1.1);
        }}
        
        .stats-content-wrapper {{
            padding: 15px;
            max-height: 500px;
            overflow-y: auto;
        }}
        .stats-panel.minimized .stats-content-wrapper {{
            display: none;
        }}
        
        .stats-subtitle {{
            font-size: 11px;
            color: #8B949E;
            text-align: center;
            margin-bottom: 12px;
            padding-bottom: 10px;
            border-bottom: 1px solid #30363D;
        }}
        .stat-row {{
            display: flex;
            justify-content: space-between;
            padding: 7px 0;
            border-bottom: 1px solid #30363D;
        }}
        .stat-label {{
            color: #8B949E;
            font-size: 12px;
        }}
        .stat-value {{
            color: #2ECC71;
            font-weight: bold;
            font-size: 13px;
        }}
        .stat-value.highlight {{
            color: #F4A261;
            font-size: 15px;
        }}
        .no-selection {{
            text-align: center;
            color: #8B949E;
            padding: 15px;
            font-style: italic;
            font-size: 13px;
        }}
        
        /* MODE BUTTONS */
        .mode-buttons {{
            display: flex;
            gap: 4px;
            margin-bottom: 12px;
        }}
        .mode-btn {{
            flex: 1;
            padding: 8px 4px;
            background: #21262D;
            color: white;
            border: 1px solid #30363D;
            border-radius: 5px;
            cursor: pointer;
            font-size: 10px;
            transition: all 0.3s;
        }}
        .mode-btn:hover, .mode-btn.active {{
            background: #2A9D8F;
            border-color: #2A9D8F;
        }}
        
        .clear-btn {{
            width: 100%;
            padding: 10px;
            margin-top: 12px;
            background: linear-gradient(135deg, #E63946 0%, #C53030 100%);
            color: white;
            border: none;
            border-radius: 6px;
            cursor: pointer;
            font-weight: bold;
            font-size: 13px;
            transition: all 0.3s;
        }}
        .clear-btn:hover {{
            transform: scale(1.02);
            box-shadow: 0 4px 15px rgba(230, 57, 70, 0.4);
        }}
        
        .instructions {{
            background: #21262D;
            border-radius: 8px;
            padding: 10px;
            margin-top: 12px;
            font-size: 10px;
            color: #8B949E;
        }}
        .instructions h4 {{
            color: #58A6FF;
            margin-bottom: 6px;
            font-size: 11px;
        }}
        .instructions ul {{
            list-style: none;
            padding: 0;
        }}
        .instructions li {{
            margin: 4px 0;
            padding-left: 12px;
            position: relative;
        }}
        .instructions li:before {{
            content: "•";
            position: absolute;
            left: 0;
            color: #2ECC71;
        }}
        
        .borough-breakdown {{
            margin-top: 12px;
            padding-top: 12px;
            border-top: 2px solid #30363D;
        }}
        .borough-breakdown h4 {{
            color: #58A6FF;
            font-size: 12px;
            margin-bottom: 8px;
        }}
        .borough-bar {{
            display: flex;
            align-items: center;
            margin: 4px 0;
            font-size: 10px;
        }}
        .borough-name {{
            width: 75px;
            color: #C9D1D9;
        }}
        .borough-bar-fill {{
            height: 10px;
            border-radius: 3px;
            margin: 0 6px;
            min-width: 4px;
            transition: width 0.3s;
        }}
        .borough-count {{
            color: #8B949E;
            min-width: 40px;
        }}
        
        /* POSITION INDICATOR */
        .position-indicator {{
            position: fixed;
            bottom: 10px;
            left: 10px;
            background: rgba(0,0,0,0.7);
            padding: 5px 10px;
            border-radius: 5px;
            font-size: 10px;
            color: #8B949E;
            z-index: 999;
        }}
        
        /* RESET POSITION BUTTON */
        .reset-pos-btn {{
            position: fixed;
            bottom: 10px;
            right: 10px;
            background: #21262D;
            border: 1px solid #30363D;
            color: #8B949E;
            padding: 8px 12px;
            border-radius: 5px;
            cursor: pointer;
            font-size: 11px;
            z-index: 999;
            transition: all 0.3s;
        }}
        .reset-pos-btn:hover {{
            background: #30363D;
            color: white;
        }}
        
        /* CUSTOM LEGEND */
        .custom-legend {{
            position: fixed;
            bottom: 50px;
            left: 20px;
            background: rgba(22, 27, 34, 0.95);
            border: 1px solid #30363D;
            border-radius: 8px;
            padding: 10px 12px;
            z-index: 999;
        }}
        .custom-legend h4 {{
            color: #C9D1D9;
            font-size: 11px;
            margin-bottom: 8px;
            font-weight: 600;
        }}
        .legend-item {{
            display: flex;
            align-items: center;
            margin: 4px 0;
            font-size: 10px;
            color: #8B949E;
        }}
        .legend-dot {{
            width: 10px;
            height: 10px;
            border-radius: 50%;
            margin-right: 8px;
        }}
        
        /* Scrollbar styling */
        .stats-content-wrapper::-webkit-scrollbar {{
            width: 6px;
        }}
        .stats-content-wrapper::-webkit-scrollbar-track {{
            background: #21262D;
            border-radius: 3px;
        }}
        .stats-content-wrapper::-webkit-scrollbar-thumb {{
            background: #30363D;
            border-radius: 3px;
        }}
        .stats-content-wrapper::-webkit-scrollbar-thumb:hover {{
            background: #484F58;
        }}
    </style>
</head>
<body>
    <div class="map-container">
        {fig_html}
    </div>
    
    <!-- MOVABLE STATS PANEL -->
    <div class="stats-panel" id="stats-panel">
        <div class="drag-handle" id="drag-handle">
            <div class="drag-handle-title">
                <span>📊</span>
                <span>Selection Stats</span>
            </div>
            <div style="display: flex; gap: 8px; align-items: center;">
                <div class="drag-dots">
                    <div><span></span><span></span></div>
                    <div><span></span><span></span></div>
                </div>
                <button class="minimize-btn" onclick="toggleMinimize(event)" title="Minimize">−</button>
            </div>
        </div>
        
        <div class="stats-content-wrapper">
            <div class="stats-subtitle">🖱️ Drag header to move • Select area for stats</div>
            
            <div class="mode-buttons">
                <button class="mode-btn" onclick="setMode('zoom')">🔍 Zoom</button>
                <button class="mode-btn" onclick="setMode('pan')">✋ Pan</button>
                <button class="mode-btn active" onclick="setMode('select')">⬜ Box</button>
                <button class="mode-btn" onclick="setMode('lasso')">〰️ Lasso</button>
            </div>
            
            <div id="stats-content">
                <div class="no-selection">
                    Use Box or Lasso select on the map to analyze properties
                </div>
            </div>
            
            <button class="clear-btn" onclick="clearSelection()">🔄 Reset Selection</button>
            
            <div class="instructions">
                <h4>📌 Quick Guide:</h4>
                <ul>
                    <li><b>Move Panel:</b> Drag the header</li>
                    <li><b>Box Select:</b> Click & drag rectangle</li>
                    <li><b>Lasso:</b> Draw freeform shape</li>
                    <li><b>Reset View:</b> Double-click map</li>
                </ul>
            </div>
        </div>
    </div>
    
    <button class="reset-pos-btn" onclick="resetPanelPosition()">📍 Reset Panel Position</button>
    
    <script>
        // =====================================================================
        // DRAGGABLE PANEL FUNCTIONALITY
        // =====================================================================
        
        const panel = document.getElementById('stats-panel');
        const handle = document.getElementById('drag-handle');
        
        let isDragging = false;
        let currentX;
        let currentY;
        let initialX;
        let initialY;
        let xOffset = 0;
        let yOffset = 0;
        
        // Load saved position from localStorage
        const savedPos = localStorage.getItem('statsPanelPos');
        if (savedPos) {{
            const pos = JSON.parse(savedPos);
            panel.style.right = 'auto';
            panel.style.left = pos.x + 'px';
            panel.style.top = pos.y + 'px';
            xOffset = pos.x;
            yOffset = pos.y;
        }}
        
        handle.addEventListener('mousedown', dragStart);
        document.addEventListener('mousemove', drag);
        document.addEventListener('mouseup', dragEnd);
        
        // Touch support
        handle.addEventListener('touchstart', dragStart);
        document.addEventListener('touchmove', drag);
        document.addEventListener('touchend', dragEnd);
        
        function dragStart(e) {{
            if (e.type === 'touchstart') {{
                initialX = e.touches[0].clientX - xOffset;
                initialY = e.touches[0].clientY - yOffset;
            }} else {{
                initialX = e.clientX - xOffset;
                initialY = e.clientY - yOffset;
            }}
            
            if (e.target === handle || handle.contains(e.target)) {{
                isDragging = true;
                panel.classList.add('dragging');
            }}
        }}
        
        function drag(e) {{
            if (isDragging) {{
                e.preventDefault();
                
                if (e.type === 'touchmove') {{
                    currentX = e.touches[0].clientX - initialX;
                    currentY = e.touches[0].clientY - initialY;
                }} else {{
                    currentX = e.clientX - initialX;
                    currentY = e.clientY - initialY;
                }}
                
                xOffset = currentX;
                yOffset = currentY;
                
                // Constrain to viewport
                const maxX = window.innerWidth - panel.offsetWidth - 10;
                const maxY = window.innerHeight - panel.offsetHeight - 10;
                
                xOffset = Math.max(10, Math.min(xOffset, maxX));
                yOffset = Math.max(10, Math.min(yOffset, maxY));
                
                panel.style.right = 'auto';
                panel.style.left = xOffset + 'px';
                panel.style.top = yOffset + 'px';
            }}
        }}
        
        function dragEnd(e) {{
            if (isDragging) {{
                isDragging = false;
                panel.classList.remove('dragging');
                
                // Save position to localStorage
                localStorage.setItem('statsPanelPos', JSON.stringify({{
                    x: xOffset,
                    y: yOffset
                }}));
            }}
        }}
        
        function resetPanelPosition() {{
            panel.style.right = '20px';
            panel.style.left = 'auto';
            panel.style.top = '80px';
            xOffset = window.innerWidth - panel.offsetWidth - 20;
            yOffset = 80;
            localStorage.removeItem('statsPanelPos');
        }}
        
        function toggleMinimize(e) {{
            e.stopPropagation();
            panel.classList.toggle('minimized');
            const btn = e.target;
            btn.textContent = panel.classList.contains('minimized') ? '+' : '−';
        }}
        
        // =====================================================================
        // MAP SELECTION FUNCTIONALITY - FIXED VERSION
        // =====================================================================
        
        const propertyData = {json.dumps(stats_data)};
        
        const boroughColors = {{
            'Manhattan': '#E63946',
            'Brooklyn': '#457B9D',
            'Queens': '#2A9D8F',
            'Bronx': '#9B59B6',
            'Staten Island': '#F4A261'
        }};
        
        const plotElement = document.getElementById('plotly-map');
        
        function setMode(mode) {{
            document.querySelectorAll('.mode-btn').forEach(btn => btn.classList.remove('active'));
            event.target.classList.add('active');
            
            let dragmode = 'zoom';
            if (mode === 'select') dragmode = 'select';
            else if (mode === 'lasso') dragmode = 'lasso';
            else if (mode === 'pan') dragmode = 'pan';
            
            Plotly.relayout(plotElement, {{'dragmode': dragmode}});
        }}
        
        function clearSelection() {{
            Plotly.restyle(plotElement, {{'selectedpoints': null}});
            document.getElementById('stats-content').innerHTML = `
                <div class="no-selection">
                    Use Box or Lasso select on the map to analyze properties
                </div>
            `;
            Plotly.restyle(plotElement, {{'marker.opacity': 0.7}});
        }}
        
        function formatCurrency(value) {{
            if (value >= 1000000) {{
                return '$' + (value / 1000000).toFixed(2) + 'M';
            }} else if (value >= 1000) {{
                return '$' + (value / 1000).toFixed(0) + 'K';
            }}
            return '$' + value.toFixed(0);
        }}
        
        function calculateStats(selectedIndices) {{
            if (!selectedIndices || selectedIndices.length === 0) return null;
            
            // FIXED: Directly use the indices to get data from propertyData array
            const selectedData = selectedIndices
                .filter(i => i >= 0 && i < propertyData.length)
                .map(i => propertyData[i])
                .filter(d => d);
            
            if (selectedData.length === 0) return null;
            
            const prices = selectedData.map(d => d.sale_price).filter(p => p > 0);
            const ppsf = selectedData.map(d => d.price_per_sqft).filter(p => p && p > 0);
            const sizes = selectedData.map(d => d.bldgarea).filter(s => s && s > 0);
            const ages = selectedData.map(d => d.building_age).filter(a => a && a > 0);
            
            const boroughCounts = {{}};
            selectedData.forEach(d => {{
                const borough = d.borough_name;
                if (borough) {{
                    boroughCounts[borough] = (boroughCounts[borough] || 0) + 1;
                }}
            }});
            
            return {{
                count: selectedData.length,
                avgPrice: prices.length > 0 ? prices.reduce((a, b) => a + b, 0) / prices.length : 0,
                medianPrice: prices.length > 0 ? prices.sort((a,b) => a-b)[Math.floor(prices.length/2)] : 0,
                minPrice: prices.length > 0 ? Math.min(...prices) : 0,
                maxPrice: prices.length > 0 ? Math.max(...prices) : 0,
                avgPPSF: ppsf.length > 0 ? ppsf.reduce((a, b) => a + b, 0) / ppsf.length : 0,
                medianPPSF: ppsf.length > 0 ? ppsf.sort((a,b) => a-b)[Math.floor(ppsf.length/2)] : 0,
                avgSize: sizes.length > 0 ? sizes.reduce((a, b) => a + b, 0) / sizes.length : 0,
                avgAge: ages.length > 0 ? ages.reduce((a, b) => a + b, 0) / ages.length : 0,
                boroughCounts: boroughCounts,
                totalValue: prices.reduce((a, b) => a + b, 0)
            }};
        }}
        
        function updateStatsDisplay(stats) {{
            if (!stats) {{
                document.getElementById('stats-content').innerHTML = `
                    <div class="no-selection">No properties selected. Try selecting an area.</div>
                `;
                return;
            }}
            
            const maxCount = Math.max(...Object.values(stats.boroughCounts));
            let boroughHTML = '';
            for (const [borough, count] of Object.entries(stats.boroughCounts).sort((a,b) => b[1] - a[1])) {{
                const width = (count / maxCount) * 80;
                const color = boroughColors[borough] || '#888';
                boroughHTML += `
                    <div class="borough-bar">
                        <span class="borough-name">${{borough}}</span>
                        <div class="borough-bar-fill" style="width: ${{width}}px; background: ${{color}};"></div>
                        <span class="borough-count">${{count.toLocaleString()}}</span>
                    </div>
                `;
            }}
            
            document.getElementById('stats-content').innerHTML = `
                <div class="stat-row">
                    <span class="stat-label">🏠 Properties</span>
                    <span class="stat-value highlight">${{stats.count.toLocaleString()}}</span>
                </div>
                <div class="stat-row">
                    <span class="stat-label">💰 Total Value</span>
                    <span class="stat-value">${{formatCurrency(stats.totalValue)}}</span>
                </div>
                <div class="stat-row">
                    <span class="stat-label">📊 Avg Price</span>
                    <span class="stat-value">${{formatCurrency(stats.avgPrice)}}</span>
                </div>
                <div class="stat-row">
                    <span class="stat-label">📈 Median Price</span>
                    <span class="stat-value">${{formatCurrency(stats.medianPrice)}}</span>
                </div>
                <div class="stat-row">
                    <span class="stat-label">⬇️ Min</span>
                    <span class="stat-value">${{formatCurrency(stats.minPrice)}}</span>
                </div>
                <div class="stat-row">
                    <span class="stat-label">⬆️ Max</span>
                    <span class="stat-value">${{formatCurrency(stats.maxPrice)}}</span>
                </div>
                <div class="stat-row">
                    <span class="stat-label">📐 Avg $/SqFt</span>
                    <span class="stat-value">${{stats.avgPPSF > 0 ? '$' + stats.avgPPSF.toFixed(0) : 'N/A'}}</span>
                </div>
                <div class="stat-row">
                    <span class="stat-label">📏 Avg Size</span>
                    <span class="stat-value">${{stats.avgSize > 0 ? Math.round(stats.avgSize).toLocaleString() + ' sf' : 'N/A'}}</span>
                </div>
                <div class="stat-row">
                    <span class="stat-label">🏛️ Avg Age</span>
                    <span class="stat-value">${{stats.avgAge > 0 ? stats.avgAge.toFixed(0) + ' yrs' : 'N/A'}}</span>
                </div>
                <div class="borough-breakdown">
                    <h4>🏙️ Borough Breakdown</h4>
                    ${{boroughHTML}}
                </div>
            `;
        }}
        
        // FIXED: Listen for selection events on all traces
        plotElement.on('plotly_selected', function(eventData) {{
            if (!eventData || !eventData.points || eventData.points.length === 0) {{
                updateStatsDisplay(null);
                return;
            }}
            
            // FIXED: Get point indices directly - works because we use single trace
            const selectedIndices = eventData.points.map(pt => pt.pointIndex);
            
            console.log('Selected indices:', selectedIndices.length, 'points');
            
            const stats = calculateStats(selectedIndices);
            updateStatsDisplay(stats);
        }});
        
        plotElement.on('plotly_deselect', function() {{
            clearSelection();
        }});
        
        Plotly.relayout(plotElement, {{'dragmode': 'select'}});
    </script>
</body>
</html>
    '''
    
    filepath = os.path.join(MAPS_DIR, filename)
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(full_html)
    
    print(f"  ✓ {filename}")
    return filepath


# =============================================================================
# 🗺️ MAP 1: BOROUGH OVERVIEW WITH SELECTION (FIXED - SINGLE TRACE)
# =============================================================================

def create_borough_overview_map(df_map):
    """
    Create borough overview map with selection and stats.
    
    FIXED: Uses SINGLE TRACE with color array so point indices match data indices.
    """
    
    # Sample the data
    sample = df_map.sample(min(15000, len(df_map)), random_state=42).reset_index(drop=True)
    
    fig = go.Figure()
    
    # Create color array based on borough
    colors = [BOROUGH_COLORS.get(b, '#888888') for b in sample['borough_name']]
    
    # Create hover texts
    hover_texts = []
    for _, row in sample.iterrows():
        text = f"<b>{row['borough_name']}</b><br>"
        text += f"Price: ${row['sale_price']:,.0f}<br>"
        text += f"Category: {row['building_category']}"
        if pd.notna(row.get('price_per_sqft')) and row['price_per_sqft'] > 0:
            text += f"<br>$/SqFt: ${row['price_per_sqft']:,.0f}"
        hover_texts.append(text)
    
    # FIXED: Single trace with all data points
    fig.add_trace(go.Scattermapbox(
        lon=sample['longitude'],
        lat=sample['latitude'],
        mode='markers',
        name='Properties',
        marker=dict(
            size=7,
            color=colors,
            opacity=0.7
        ),
        text=hover_texts,
        hovertemplate='%{text}<extra></extra>',
        selected=dict(marker=dict(opacity=1, size=10)),
        unselected=dict(marker=dict(opacity=0.1, size=5)),
    ))
    
    # Create custom legend using annotations
    borough_counts = sample['borough_name'].value_counts()
    
    fig.update_layout(
        title=dict(
            text='<b>🗽 NYC Property Sales - Select Areas to Analyze</b>',
            font=dict(size=16),
            x=0.5
        ),
        mapbox=dict(
            style='carto-darkmatter',
            center=dict(lat=40.7128, lon=-73.9700),
            zoom=9.5,
        ),
        height=MAP_HEIGHT,
        width=MAP_WIDTH,
        showlegend=False,  # We'll use custom legend
        margin=dict(l=0, r=0, t=50, b=0),
        paper_bgcolor='#0D1117',
        font=dict(color='white'),
        dragmode='select',
        # Add custom legend as annotations
        annotations=[
            dict(
                x=0.02, y=0.98, xref='paper', yref='paper',
                text='<b>🏙️ Boroughs</b>', showarrow=False,
                font=dict(size=12, color='white'),
                bgcolor='rgba(0,0,0,0.7)', borderpad=4,
                xanchor='left', yanchor='top'
            ),
            dict(
                x=0.02, y=0.93, xref='paper', yref='paper',
                text=f"<span style='color:#E63946'>●</span> Manhattan ({borough_counts.get('Manhattan', 0):,})",
                showarrow=False, font=dict(size=10, color='white'),
                bgcolor='rgba(0,0,0,0.7)', borderpad=4,
                xanchor='left', yanchor='top'
            ),
            dict(
                x=0.02, y=0.88, xref='paper', yref='paper',
                text=f"<span style='color:#457B9D'>●</span> Brooklyn ({borough_counts.get('Brooklyn', 0):,})",
                showarrow=False, font=dict(size=10, color='white'),
                bgcolor='rgba(0,0,0,0.7)', borderpad=4,
                xanchor='left', yanchor='top'
            ),
            dict(
                x=0.02, y=0.83, xref='paper', yref='paper',
                text=f"<span style='color:#2A9D8F'>●</span> Queens ({borough_counts.get('Queens', 0):,})",
                showarrow=False, font=dict(size=10, color='white'),
                bgcolor='rgba(0,0,0,0.7)', borderpad=4,
                xanchor='left', yanchor='top'
            ),
            dict(
                x=0.02, y=0.78, xref='paper', yref='paper',
                text=f"<span style='color:#9B59B6'>●</span> Bronx ({borough_counts.get('Bronx', 0):,})",
                showarrow=False, font=dict(size=10, color='white'),
                bgcolor='rgba(0,0,0,0.7)', borderpad=4,
                xanchor='left', yanchor='top'
            ),
            dict(
                x=0.02, y=0.73, xref='paper', yref='paper',
                text=f"<span style='color:#F4A261'>●</span> Staten Is. ({borough_counts.get('Staten Island', 0):,})",
                showarrow=False, font=dict(size=10, color='white'),
                bgcolor='rgba(0,0,0,0.7)', borderpad=4,
                xanchor='left', yanchor='top'
            ),
        ]
    )
    
    # Pass the same sample data for stats calculation
    create_map_with_selection_stats(fig, 'interactive_map_01_borough_overview.html', sample)
    return fig

create_borough_overview_map(df_map)

# =============================================================================
# 🗺️ MAP 2: PRICE PER SQFT WITH SELECTION (FIXED)
# =============================================================================

def create_price_per_sqft_map(df_res_map):
    """Create price per sqft map with selection and stats - FIXED."""
    
    sample = df_res_map[df_res_map['price_per_sqft'].notna()].copy()
    if len(sample) > 10000:
        sample = sample.sample(10000, random_state=42)
    sample = sample.reset_index(drop=True)
    
    fig = go.Figure()
    
    hover_texts = []
    for _, row in sample.iterrows():
        text = f"<b>{row['borough_name']}</b><br>"
        text += f"$/SqFt: ${row['price_per_sqft']:,.0f}<br>"
        text += f"Price: ${row['sale_price']:,.0f}<br>"
        text += f"Size: {row['bldgarea']:,.0f} sqft"
        hover_texts.append(text)
    
    # Single trace
    fig.add_trace(go.Scattermapbox(
        lon=sample['longitude'],
        lat=sample['latitude'],
        mode='markers',
        marker=dict(
            size=8,
            color=sample['price_per_sqft'],
            colorscale='YlOrRd',
            cmin=sample['price_per_sqft'].quantile(0.05),
            cmax=sample['price_per_sqft'].quantile(0.95),
            opacity=0.7,
            colorbar=dict(
                title=dict(text='$/SqFt', font=dict(size=11, color='white')),
                tickfont=dict(color='white', size=9),
                len=0.5, thickness=15, x=0.98,
            )
        ),
        text=hover_texts,
        hovertemplate='%{text}<extra></extra>',
        selected=dict(marker=dict(opacity=1, size=12)),
        unselected=dict(marker=dict(opacity=0.1, size=5)),
    ))
    
    fig.update_layout(
        title=dict(text='<b>💰 Price per SqFt - Select Areas to Analyze</b>', font=dict(size=16), x=0.5),
        mapbox=dict(style='carto-darkmatter', center=dict(lat=40.7128, lon=-73.9700), zoom=9.5),
        height=MAP_HEIGHT, width=MAP_WIDTH,
        margin=dict(l=0, r=60, t=50, b=0),
        paper_bgcolor='#0D1117', font=dict(color='white'),
        dragmode='select',
    )
    
    create_map_with_selection_stats(fig, 'interactive_map_02_price_per_sqft.html', sample)
    return fig

create_price_per_sqft_map(df_res_map)

# =============================================================================
# 🗺️ MAP 3: SALE PRICE WITH SELECTION (FIXED)
# =============================================================================

def create_sale_price_map(df_map):
    """Create sale price map with selection and stats - FIXED."""
    
    sample = df_map[df_map['sale_price'] < 5000000].copy()
    if len(sample) > 10000:
        sample = sample.sample(10000, random_state=42)
    sample = sample.reset_index(drop=True)
    
    fig = go.Figure()
    
    hover_texts = []
    for _, row in sample.iterrows():
        text = f"<b>{row['borough_name']}</b><br>"
        text += f"Price: ${row['sale_price']:,.0f}<br>"
        text += f"Category: {row['building_category']}"
        hover_texts.append(text)
    
    # Single trace
    fig.add_trace(go.Scattermapbox(
        lon=sample['longitude'],
        lat=sample['latitude'],
        mode='markers',
        marker=dict(
            size=8,
            color=sample['sale_price'],
            colorscale='Viridis',
            cmin=sample['sale_price'].quantile(0.05),
            cmax=sample['sale_price'].quantile(0.95),
            opacity=0.7,
            colorbar=dict(
                title=dict(text='Price ($)', font=dict(size=11, color='white')),
                tickfont=dict(color='white', size=9),
                len=0.5, thickness=15, x=0.98,
            )
        ),
        text=hover_texts,
        hovertemplate='%{text}<extra></extra>',
        selected=dict(marker=dict(opacity=1, size=12)),
        unselected=dict(marker=dict(opacity=0.1, size=5)),
    ))
    
    fig.update_layout(
        title=dict(text='<b>🏠 Sale Price - Select Areas to Analyze</b>', font=dict(size=16), x=0.5),
        mapbox=dict(style='carto-darkmatter', center=dict(lat=40.7128, lon=-73.9700), zoom=9.5),
        height=MAP_HEIGHT, width=MAP_WIDTH,
        margin=dict(l=0, r=60, t=50, b=0),
        paper_bgcolor='#0D1117', font=dict(color='white'),
        dragmode='select',
    )
    
    create_map_with_selection_stats(fig, 'interactive_map_03_sale_price.html', sample)
    return fig

create_sale_price_map(df_map)

# =============================================================================
# 🗺️ MAP 4: BUILDING AGE WITH SELECTION (FIXED)
# =============================================================================

def create_building_age_map(df_map):
    """Create building age map with selection and stats - FIXED."""
    
    sample = df_map[df_map['building_age'].notna()].copy()
    if len(sample) > 10000:
        sample = sample.sample(10000, random_state=42)
    sample = sample.reset_index(drop=True)
    
    fig = go.Figure()
    
    hover_texts = []
    for _, row in sample.iterrows():
        text = f"<b>{row['borough_name']}</b><br>"
        text += f"Age: {int(row['building_age'])} years<br>"
        text += f"Built: {int(row['yearbuilt'])}<br>"
        text += f"Price: ${row['sale_price']:,.0f}"
        hover_texts.append(text)
    
    # Single trace
    fig.add_trace(go.Scattermapbox(
        lon=sample['longitude'],
        lat=sample['latitude'],
        mode='markers',
        marker=dict(
            size=8,
            color=sample['building_age'],
            colorscale='Plasma',
            cmin=0, cmax=120,
            opacity=0.7,
            colorbar=dict(
                title=dict(text='Age (yrs)', font=dict(size=11, color='white')),
                tickfont=dict(color='white', size=9),
                len=0.5, thickness=15, x=0.98,
            )
        ),
        text=hover_texts,
        hovertemplate='%{text}<extra></extra>',
        selected=dict(marker=dict(opacity=1, size=12)),
        unselected=dict(marker=dict(opacity=0.1, size=5)),
    ))
    
    fig.update_layout(
        title=dict(text='<b>🏛️ Building Age - Select Areas to Analyze</b>', font=dict(size=16), x=0.5),
        mapbox=dict(style='carto-darkmatter', center=dict(lat=40.7128, lon=-73.9700), zoom=9.5),
        height=MAP_HEIGHT, width=MAP_WIDTH,
        margin=dict(l=0, r=60, t=50, b=0),
        paper_bgcolor='#0D1117', font=dict(color='white'),
        dragmode='select',
    )
    
    create_map_with_selection_stats(fig, 'interactive_map_04_building_age.html', sample)
    return fig

create_building_age_map(df_map)

# =============================================================================
# 🗺️ MAP 5: PRICE TIERS WITH SELECTION (FIXED - SINGLE TRACE)
# =============================================================================

def create_price_tier_map(df_map):
    """Create price tier map with selection and stats - FIXED with single trace."""
    
    sample = df_map.sample(min(15000, len(df_map)), random_state=42).reset_index(drop=True)
    
    fig = go.Figure()
    
    price_colors_map = {
        'Under $300K': '#2ECC71', '$300K-600K': '#3498DB', '$600K-900K': '#9B59B6',
        '$900K-1.5M': '#E74C3C', '$1.5M-3M': '#E67E22', 'Over $3M': '#F1C40F'
    }
    
    # Create color array based on price tier
    colors = [price_colors_map.get(t, '#888888') for t in sample['price_tier']]
    
    hover_texts = []
    for _, row in sample.iterrows():
        text = f"<b>{row.get('price_tier', 'Unknown')}</b><br>"
        text += f"Price: ${row['sale_price']:,.0f}<br>"
        text += f"Borough: {row['borough_name']}"
        hover_texts.append(text)
    
    # FIXED: Single trace with all data
    fig.add_trace(go.Scattermapbox(
        lon=sample['longitude'],
        lat=sample['latitude'],
        mode='markers',
        name='Properties',
        marker=dict(
            size=6,
            color=colors,
            opacity=0.7
        ),
        text=hover_texts,
        hovertemplate='%{text}<extra></extra>',
        selected=dict(marker=dict(opacity=1, size=10)),
        unselected=dict(marker=dict(opacity=0.1, size=4)),
    ))
    
    # Custom legend as annotations
    tier_order = ['Under $300K', '$300K-600K', '$600K-900K', '$900K-1.5M', '$1.5M-3M', 'Over $3M']
    tier_counts = sample['price_tier'].value_counts()
    
    annotations = [
        dict(
            x=0.02, y=0.98, xref='paper', yref='paper',
            text='<b>💵 Price Tiers</b>', showarrow=False,
            font=dict(size=12, color='white'),
            bgcolor='rgba(0,0,0,0.7)', borderpad=4,
            xanchor='left', yanchor='top'
        )
    ]
    
    for i, tier in enumerate(tier_order):
        color = price_colors_map[tier]
        count = tier_counts.get(tier, 0)
        annotations.append(
            dict(
                x=0.02, y=0.93 - i*0.05, xref='paper', yref='paper',
                text=f"<span style='color:{color}'>●</span> {tier} ({count:,})",
                showarrow=False, font=dict(size=10, color='white'),
                bgcolor='rgba(0,0,0,0.7)', borderpad=4,
                xanchor='left', yanchor='top'
            )
        )
    
    fig.update_layout(
        title=dict(text='<b>💵 Price Tiers - Select Areas to Analyze</b>', font=dict(size=16), x=0.5),
        mapbox=dict(style='carto-darkmatter', center=dict(lat=40.7128, lon=-73.9700), zoom=9.5),
        height=MAP_HEIGHT, width=MAP_WIDTH,
        showlegend=False,
        annotations=annotations,
        margin=dict(l=0, r=0, t=50, b=0),
        paper_bgcolor='#0D1117', font=dict(color='white'),
        dragmode='select',
    )
    
    create_map_with_selection_stats(fig, 'interactive_map_05_price_tiers.html', sample)
    return fig

create_price_tier_map(df_map)

# =============================================================================
# 🗺️ BOROUGH DETAIL MAPS WITH SELECTION (FIXED)
# =============================================================================

def create_borough_detail_map(df_map, df_res_map, borough):
    """Create borough detail map with selection and stats - FIXED."""
    
    borough_all = df_map[df_map['borough_name'] == borough]
    borough_res = df_res_map[df_res_map['borough_name'] == borough]
    
    if len(borough_all) == 0:
        return
    
    center_lat = borough_all['latitude'].mean()
    center_lon = borough_all['longitude'].mean()
    
    # Get properties with valid price per sqft
    valid_pps = borough_res[borough_res['price_per_sqft'].notna()].copy()
    valid_pps = valid_pps.reset_index(drop=True)  # Reset index for proper alignment
    
    fig = go.Figure()
    
    if len(valid_pps) > 0:
        hover_texts = []
        for _, r in valid_pps.iterrows():
            text = f"<b>{borough}</b><br>"
            text += f"$/SqFt: ${r['price_per_sqft']:,.0f}<br>"
            text += f"Price: ${r['sale_price']:,.0f}<br>"
            text += f"Size: {r['bldgarea']:,.0f} sqft"
            hover_texts.append(text)
        
        # Single trace
        fig.add_trace(go.Scattermapbox(
            lon=valid_pps['longitude'],
            lat=valid_pps['latitude'],
            mode='markers',
            marker=dict(
                size=10,
                color=valid_pps['price_per_sqft'],
                colorscale='YlOrRd',
                cmin=valid_pps['price_per_sqft'].quantile(0.05),
                cmax=valid_pps['price_per_sqft'].quantile(0.95),
                opacity=0.8,
                colorbar=dict(
                    title=dict(text='$/SqFt', font=dict(size=11, color='white')),
                    tickfont=dict(color='white', size=9),
                    len=0.5, thickness=15, x=0.98
                )
            ),
            text=hover_texts,
            hovertemplate='%{text}<extra></extra>',
            selected=dict(marker=dict(opacity=1, size=14)),
            unselected=dict(marker=dict(opacity=0.1, size=6)),
        ))
    
    median_price = borough_all['sale_price'].median()
    median_pps = valid_pps['price_per_sqft'].median() if len(valid_pps) > 0 else 0
    
    fig.update_layout(
        title=dict(
            text=f'<b>🏙️ {borough}</b> | ${median_price/1000:.0f}K median | ${median_pps:.0f}/sqft',
            font=dict(size=14),
            x=0.5
        ),
        mapbox=dict(
            style='carto-darkmatter',
            center=dict(lat=center_lat, lon=center_lon),
            zoom=10.5
        ),
        height=MAP_HEIGHT,
        width=MAP_WIDTH,
        margin=dict(l=0, r=60, t=40, b=0),
        paper_bgcolor='#0D1117',
        font=dict(color='white'),
        dragmode='select',
    )
    
    create_map_with_selection_stats(
        fig, 
        f'interactive_borough_{borough.lower().replace(" ", "_")}.html', 
        valid_pps
    )
    return fig

# Create all borough maps
for borough in ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island']:
    create_borough_detail_map(df_map, df_res_map, borough)

# =============================================================================
# SUMMARY
# =============================================================================

print("\n" + "="*80)
print("🎉 INTERACTIVE MAPS WITH FIXED SELECTION STATS COMPLETE!")
print("="*80)
print("""
✅ FIXED ISSUES:
  • Point indices now correctly match data array indices
  • Single trace approach ensures proper selection alignment
  • Borough breakdown will accurately reflect selected area

📍 FEATURES:
  🖱️ DRAG & DROP - Move the stats panel anywhere on screen!
  📌 POSITION MEMORY - Panel remembers where you left it
  ➖ MINIMIZE - Click minus button to collapse panel
  🔄 RESET POSITION - Button to restore default position

📊 Selection Stats Available:
  • Properties count & total value
  • Average/Median prices
  • Price range (min/max)
  • Average $/SqFt
  • Average size & building age
  • Borough breakdown (NOW ACCURATE!)

📁 Generated 10 Interactive Maps in 'maps/' folder:
  • interactive_map_01_borough_overview.html
  • interactive_map_02_price_per_sqft.html
  • interactive_map_03_sale_price.html
  • interactive_map_04_building_age.html
  • interactive_map_05_price_tiers.html
  • interactive_borough_manhattan.html
  • interactive_borough_brooklyn.html
  • interactive_borough_queens.html
  • interactive_borough_bronx.html
  • interactive_borough_staten_island.html
""")